In [1]:
import plotly.express as px
import pandas as pd
import pickle

In [2]:
df = pickle.load(open("data/loans_clean.pkd", "rb"))

df = df[df['State'] != "XX"]
df = df[df['State'] != "AS"]
df = df[df['State'] != "MP"]
df = df[df['State'] != "GU"]
df = df[df['State'] != "PR"]
df = df[df['State'] != "DC"]
df = df[df['State'] != "VI"]

In [3]:
states = list(df['State'].unique())
cats = list(df['LoanCat'].unique())

In [4]:
loan_count_dict = {}
for state in states:
    temp = df[df['State'] == state]
    loan_count_dict[state] = {}
    for lclass in cats:
        try:
            loan_count_dict[state][lclass] = temp.groupby('LoanCat')['LoanCat'].count()[lclass]
        except (AttributeError and KeyError):
            loan_count_dict[state][cat] = 0

In [5]:
# pickle.dump(dict, open("data/loan_counts_dict.pkd", "wb"))

In [4]:
loan_count_dict = pickle.load(open("data/loan_counts_dict.pkd", "rb"))

In [5]:
df = pd.DataFrame.from_dict(loan_count_dict, orient="index")

In [6]:
df.reset_index(level=0, inplace=True)

In [7]:
df = df.rename(columns = {"index":"State"})

In [8]:
loan_vals = {
    "a" : 7.5,
    "b" : 3.5,
    "c" : 1.5,
    "d" : 0.675,
    "e" : 0.25
}

In [9]:
# for lclass, lval in list(zip((loan_vals.keys(), loan_vals.values()))):
for (lclass, lval) in loan_vals.items():
    df[str(lclass) + "_sum"] = df.apply(lambda row: lval * row[str(lclass)], axis=1)

In [10]:
df['loan_sum'] = df.apply(lambda row: row['a_sum'] + row['b_sum'] + row['c_sum'] + row['d_sum'] + row['e_sum'], axis=1)

In [11]:
df = df.sort_values(by=['loan_sum'])

In [21]:
pickle.dump(df, open("data/loan_sum_by_state.pkd", "wb"))

In [15]:
df = pickle.load(open("data/loan_sum_by_state.pkd", "rb"))

In [18]:
fig = px.histogram(df, x="State", y="loan_sum",
                  title="Total Loans in Millions of Dollars per State")

In [22]:
fig.update_xaxes(title_text='State')
fig.update_yaxes(title_text='Loan Total (Millions of $)')
fig.show()

In [20]:
fig.write_html("templates/loan_sum_by_state.html", include_plotlyjs="cdn")

In [21]:
fig.write_image("loan_sum_by_state.png")